Baixando dependências

In [9]:
!pip install opencv-python
!pip install numpy
!pip install matplotlib

Importando dependências para o projeto

In [3]:
import cv2 as cv

In [4]:
captura = cv.VideoCapture('../videos/1.mp4')
if not captura.isOpened():
    print("Error al abrir video")
    exit()

while True:
    ret, frame = captura.read()
    if not ret:
        cv.destroyAllWindows()
        print("No se puede recibir frame")
        break
    cv.imshow('Frame', frame)
    if cv.waitKey(30) & 0xFF == ord('q'):
        cv.destroyAllWindows()
        break

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/bootwin-pc/.pyenv/versions/3.12.6/lib/python3.12/site-packages/cv2/qt/plugins"


AttributeError: module 'cv2' has no attribute 'CloseAllWindows'